In [1]:
import pandas as pd

# generate state code and name dictionary
North: JK, Ladakh, PN, HP, HR, UK, Delhi, Chandigarh• 
West: RJ, GJ, MH, Goa, Dadra & Nagar Haveli, Daman & Diu•  
Central: MP, UP, CG•  
East: BH, WB, OR, JH•  
South: KT, TG, AP, TN, KL, Lakshadweep, Puducherry• 
North-East: AS, SK, MG, TR, AR, MN, NG, MZ, Andaman & NicobarFor

In [115]:
North=list(range(1,8))
West=list(range(8,14))
Central=list(range(14,17))
East=list(range(17,21))
South=list(range(21,27))
North_East=list(range(27,36))

In [ ]:
def getLangs(region):
    for code in region:
        df=pd.read_excel(f'datasets/C-17/{str(code).zfill(2)}.xlsx',skiprows=6,header=None,engine='openpyxl')

In [4]:
df=pd.read_excel(f'datasets/C-17/01.xlsx',skiprows=6,header=None,engine='openpyxl')
df.fillna(value=0,inplace=True)

In [6]:
df1=pd.read_excel(f'datasets/C-17/02.xlsx',skiprows=6,header=None,engine='openpyxl')
df1.fillna(value=0,inplace=True)

In [7]:
df3=pd.concat([df,df1])

In [12]:
df3.shape

(3679, 17)

In [101]:
df=pd.read_excel(f'datasets/C-17/00.xlsx',skiprows=6,header=None,engine='openpyxl')
df.fillna(value=0,inplace=True)

In [102]:
l=df.iloc[:,3].to_list()
l.extend(df.iloc[:,8].to_list())
l.extend(df.iloc[:,13].to_list())

In [103]:
langsList=[]
for langs in l:
    if (langs!=0) and (langs not in langsList):
        langsList.append(langs)

In [104]:
len(langsList)

122

In [105]:
langCounts={}
for lang in langsList:
    langCounts[lang]=0
for i,lang in enumerate(langsList):
    for data in df.iloc[:,3].to_list():
        if data==lang:
            langCounts[lang]=langCounts[lang]+df.iloc[i,4]
    for data in df.iloc[:,8].to_list():
        if data==lang:
            langCounts[lang]=langCounts[lang]+df.iloc[i,9]
    for data in df.iloc[:,13].to_list():
        if data==lang:
            langCounts[lang]=langCounts[lang]+df.iloc[i,14]

In [106]:
df=pd.DataFrame({'name':list(langCounts.keys()),'count':list(langCounts.values())})

In [108]:
sum(list(langCounts.values()))

439520004.0

In [107]:
df.sort_values('count',ascending=False,axis=0)

,name,count
1,BENGALI,232909845.0
4,GUJARATI,106466040.0
117,YIMCHUNGRE,40905138.0
84,MARAM,30692220.0
0,ASSAMESE,15311351.0
...,...,...
57,KHOND/KONDH,39.0
58,KINNAURI,24.0
82,MALTO,22.0
32,BISHNUPURIYA,16.0


In [109]:
sum(list(langCounts.values()))/10e+6

43.9520004

In [48]:
for i,data in enumerate(langsList):
    print(i,data)

0 BENGALI 
1 BODO 
2 HINDI 
3 KASHMIRI 
4 MANIPURI 
5 NEPALI 
6 PUNJABI 
7 URDU
8 ENGLISH 
9 MIRI/MISHING
10 OTHERS
11 ASSAMESE
12 DOGRI 
13 GUJARATI
14 MARATHI 
15 ODIA 
16 SANSKRIT 
17 SANTALI 
18 TAMIL 
19 TELUGU 
20 ARABIC/ARBI 
21 TRIPURI 
22 MAITHILI 
23 MALAYALAM
24 BALTI 
25 BHOTIA 
26 KHANDESHI
27 KHASI 
28 LADAKHI 
29 LAHNDA 
30 SHINA
31 TIBETAN 
32 KANNADA
33 KONKANI
34 AFGHANI/KABULI/PASHTO 
35 ANAL 
36 COORGI/KODAGU 
37 GONDI
38 KHARIA 
39 KINNAURI 
40 KURUKH/ORAON 
41 LAHAULI 
42 LUSHAI/MIZO 
43 MUNDA
44 MUNDARI 
45 NICOBARESE 
46 TULU 
47 SINDHI
48 BHILI/BHILODI 
49 HALAM 
50 KUKI
51 PAITE
52 KONYAK 
53 ADI 
54 CHANG 
55 RABHA 
56 THADO 
57 KISAN 
